In [ ]:
import os
import sys
import numpy as np  # type: ignore
import lyncs_io as lio  # type: ignore
from typing import List

In [ ]:
import plaquette as RPlaq

In [ ]:
# the location, name and ID number of the gauefields
gfDir = 'confs'
gfName = 'Gen2_8x24n'
gfIDs = [7,8,9]

In [ ]:
# Load the gaugefields
# Make a list of gaugefield data
gfData: List[np.ndarray] = []
# Loop over each ID
for iid in gfIDs:
    gfFile = os.path.join(gfDir,f'{gfName}{iid}')
    # Load and append
    # Here we have specified the full path and the format
    # Can figure it out based on extension, but format is clearer
    gfData.append(lio.load(gfFile, format='openqcd'))
# Convert to array for better indexing
gfAr = np.asarray(gfData)

In [ ]:
# Here show the shape of the data
# it is [ID, NT, NX, NY, NZ, mu, colour, colour]
print(gfAr.shape)

In [ ]:
# Use Ryan's plaquette code to calculate whole of lattice plaquette
# the sum of plaquettes, the number of plaquttes, the average plaquette and the time taken
sumTrP, nP, ave, time = RPlaq.plaquette(gfAr[0, ...])
print(f'calculated average plaquette {ave} in {time:.2} seconds')

In [ ]:
# Use Ryan's plaquette code to calculate spatial plaquette
ssumTrP, snP, save, stime = RPlaq.plaquette(gfAr[0, ...], muStart=1, muEnd=4, nuEnd=4)
print(f'calculated average spatial plaquette {save} in {stime:.2} seconds')

In [ ]:
# Use Ryan's plaquette code to calculate temporal plaquette
tsumTrP, tnP, tave, ttime = RPlaq.plaquette(gfAr[0, ...], muStart=0, muEnd=1, nuEnd=4)
print(f'calculated average temporal plaquette {tave} in {ttime:.2} seconds')

In [ ]:
# Just check that this agrees with the whole of lattice plaquette (visually)
print(f'average of temporal and spatial plaquettes {(tave+save)/2.0}')